In [3]:
import IPython, nussl, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa, cmath,math,time,vamp
import os
from IPython.display import Audio, display
from scipy.fftpack import fft, ifft
import csv
from sklearn.neighbors import KNeighborsClassifier

/Users/olivergoodman/miniconda2/envs/eecs352/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [11]:
from sklearn.naive_bayes import GaussianNB

def trainGaussianNB(path_to_csv):
    """
    given a path to the feature vector table (csv file)
    returns a trained nearest neighbor model, the validation set and the testing (validation) target labels
    """
    counter = 0
    train = np.array([])
    test = np.array([])
    genres_train = np.array([])
    genres_test = np.array([])
    
    # get csv data
    csv = np.genfromtxt(path_to_csv, dtype='string', delimiter=',')
    
    # separate into vectors and labels
    labels = csv[:,-1]
    feature_vectors = csv[:,:-1].astype(float)
    
    train = feature_vectors[np.where(np.arange(feature_vectors.shape[0]) % 100 < 85)]
    test = feature_vectors[np.where(np.arange(feature_vectors.shape[0]) % 100 >= 85)]
    genres_train = labels[np.where(np.arange(labels.shape[0]) % 100 < 85)]
    genres_test = labels[np.where(np.arange(labels.shape[0]) % 100 >= 85)]
    
    # train csv file
    nn = GaussianNB()
    nn.fit(train, genres_train) 
    
    # output prediction accuracy
    predictions = nn.predict(test)
    count = 0
    for i, prediction in enumerate(predictions):
        if prediction == genres_test[i]:
            count += 1
    print "Prediction Accuracy: ", float(count)/len(genres_test)

    return nn, test, genres_train
    


In [12]:
gnb_control, test_gnb_control, genres_train = trainGaussianNB('features/control.csv')

Prediction Accuracy:  0.34


In [13]:
gnb_hpss_percussive, test_hpss_percussive, genres_hpss_percussive = trainGaussianNB('features/hpss_percussive.csv')

Prediction Accuracy:  0.306666666667


In [15]:
gnb_hpss_harmonic, test_hpss_harmonic, genres_hpss_harmonic = trainGaussianNB('features/hpss_harmonic.csv')

Prediction Accuracy:  0.306666666667


In [17]:
gnb_repet_fg, test_repet_fg, genres_repet_fg = trainGaussianNB('features/repet_foreground.csv')

Prediction Accuracy:  0.306666666667


In [19]:
gnb_repet_bg, test_repet_bg, genres_repet_bg = trainGaussianNB('features/repet_background.csv')

Prediction Accuracy:  0.346666666667


In [7]:
def trainNN(path_to_csv, num_neighbors):
    """
    given a path to the feature vector table (csv file)
    returns a trained nearest neighbor model, the validation set and the testing (validation) target labels
    """
    counter = 0
    train = np.array([])
    test = np.array([])
    genres_train = np.array([])
    genres_test = np.array([])
    
    # get csv data
    csv = np.genfromtxt(path_to_csv, dtype='string', delimiter=',')
    
    # separate into vectors and labels
    labels = csv[:,-1]
    feature_vectors = csv[:,:-1].astype(float)
    
    train = feature_vectors[np.where(np.arange(feature_vectors.shape[0]) % 100 < 85)]
    test = feature_vectors[np.where(np.arange(feature_vectors.shape[0]) % 100 >= 85)]
    genres_train = labels[np.where(np.arange(labels.shape[0]) % 100 < 85)]
    genres_test = labels[np.where(np.arange(labels.shape[0]) % 100 >= 85)]
    
    # train csv file
    nn = KNeighborsClassifier(n_neighbors=num_neighbors)
    nn.fit(train, genres_train) 
    
    # output prediction accuracy
    predictions = nn.predict(test)
    count = 0
    for i, prediction in enumerate(predictions):
        if prediction == genres_test[i]:
            count += 1
    print "Prediction Accuracy: ", float(count)/len(genres_test)

    return nn, test, genres_train
    

In [8]:
nn_control, test_control, genres_train_control = trainNN('features/control.csv', 4)

Prediction Accuracy:  0.293333333333


In [57]:
nn_hpss_percussive, test_hpss_percussive, genres_hpss_percussive = trainNN('features/hpss_percussive.csv', 4)

Prediction Accuracy:  0.293333333333


In [58]:
nn_hpss_harmonic, test_hpps_harmonic, genres_hpss_harmonic = trainNN('features/hpss_harmonic.csv', 4)

Prediction Accuracy:  0.306666666667


In [59]:
nn_repet_background, test_repet_background, genres_repet_background = trainNN('features/repet_background.csv', 4)

Prediction Accuracy:  0.26


In [60]:
nn_repet_foreground, test_repet_foreground, genres_repet_foreground = trainNN('features/repet_foreground.csv', 4)

Prediction Accuracy:  0.206666666667
